In [19]:
import os
import re
import shutil

In [20]:
yellow_dir = 'data/green_tripdata'

In [21]:
base_dir = os.path.dirname(yellow_dir)

In [22]:
base_dir

'data'

In [9]:
for filename in os.listdir(yellow_dir):
    match = re.match(r'green_tripdata_(\d{4})-\d{2}\.parquet', filename)
    if match:
        year = match.group(1)
        year_dir = os.path.join(base_dir, year)
        os.makedirs(year_dir, exist_ok=True)
        src_path = os.path.join(yellow_dir, filename)
        dst_path = os.path.join(year_dir, filename)
        shutil.move(src_path, dst_path)
        print(f"Moved: {filename} → {year}/")
if not os.listdir(yellow_dir):
    os.rmdir(yellow_dir)
    print("Removed empty folder: yellow_tripdata")

NameError: name 'yellow_dir' is not defined

In [25]:
from minio import Minio
from minio.error import S3Error

# Tạo client MinIO
client = Minio(
    "localhost:9000",            # hoặc endpoint MinIO của bạn
    access_key="minioadmin",
    secret_key="minioadmin123",
    secure=False                 # True nếu dùng HTTPS
)

bucket_name = "sandbox"

# ⚠️ Bucket phải trống trước khi xóa
try:
    # Xóa tất cả object trong bucket (nếu có)
    objects = client.list_objects(bucket_name, recursive=True)
    for obj in objects:
        client.remove_object(bucket_name, obj.object_name)
        print(f"Đã xoá object: {obj.object_name}")

    # Sau khi bucket trống, xóa bucket
    client.remove_bucket(bucket_name)
    print(f"✅ Đã xoá bucket: {bucket_name}")

except S3Error as e:
    print(f"❌ Lỗi khi xóa bucket: {e}")


Đã xoá object: 2024/green_tripdata_2024-01.parquet/_delta_log/00000000000000000000.json
Đã xoá object: 2024/green_tripdata_2024-01.parquet/_delta_log/00000000000000000001.json
Đã xoá object: 2024/green_tripdata_2024-01.parquet/part-00000-b7ec2607-0bca-43c4-bab2-51bfeff64980-c000.snappy.parquet
Đã xoá object: 2024/green_tripdata_2024-01.parquet/part-00000-c6807a0f-8961-4094-a46e-f2ab9d103d55-c000.snappy.parquet
Đã xoá object: 2024/green_tripdata_2024-02.parquet/_delta_log/00000000000000000000.json
Đã xoá object: 2024/green_tripdata_2024-02.parquet/_delta_log/00000000000000000001.json
Đã xoá object: 2024/green_tripdata_2024-02.parquet/part-00000-27ea4955-a158-4359-a6a8-1683ac425b60-c000.snappy.parquet
Đã xoá object: 2024/green_tripdata_2024-02.parquet/part-00000-2ba446a7-81d8-428e-bbda-12826544e2d4-c000.snappy.parquet
Đã xoá object: 2024/green_tripdata_2024-03.parquet/_delta_log/00000000000000000000.json
Đã xoá object: 2024/green_tripdata_2024-03.parquet/_delta_log/00000000000000000001.j

In [4]:
import pandas as pd
import os
import pyarrow.parquet as pq

In [5]:
def is_parquet_file(filepath):
    try:
        pq.ParquetFile(filepath)
        return True
    except Exception:
        return False

root_folder = "data"
for year in os.listdir(root_folder):
    full_path = os.path.join(root_folder, year)
    if os.path.isdir(full_path):
        for file in os.listdir(full_path):
            object_name = f'{year}/{file}'

            file_path = os.path.join(full_path,file)
            # print(object_name)
            # if file_path.endswith(".parquet") and is_parquet_file(file_path):
            #     df = pd.read_parquet(file_path, engine='pyarrow')
            #     print(df.head(3))
            # else:
            #     print(f"Skipping non-parquet or corrupted file: {file_path}")
            

AttributeError: 'str' object has no attribute 'object_name'

In [13]:
import os
import requests

In [18]:
years = [2024,2025]  # Bạn có thể thêm nhiều năm: [2020, 2021, 2022, 2023, 2024, 2025]

# Thư mục gốc chứa data
base_data_dir = "/Users/tamdang/Desktop/streaming_batching_end_to_end_pipeline/streaming_batching_end_to_end_pipeline/data"

for year in years:
    year_str = str(year)
    year_dir = os.path.join(base_data_dir, year_str)  # data/2025
    os.makedirs(year_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

    for month in range(1, 13):
        month_str = f"{month:02d}"
        file_name = f"green_tripdata_{year}-{month_str}.parquet"
        url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file_name}"
        output_path = os.path.join(year_dir, file_name)

        print(f"Tải về {file_name} vào {output_path}...")

        try:
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                with open(output_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"✔️ Tải thành công: {file_name}")
            else:
                print(f"❌ Không tìm thấy file: {file_name} (status: {response.status_code})")
        except Exception as e:
            print(f"⚠️ Lỗi khi tải {file_name}: {e}")

Tải về green_tripdata_2024-01.parquet vào /Users/tamdang/Desktop/streaming_batching_end_to_end_pipeline/streaming_batching_end_to_end_pipeline/data/2024/green_tripdata_2024-01.parquet...
✔️ Tải thành công: green_tripdata_2024-01.parquet
Tải về green_tripdata_2024-02.parquet vào /Users/tamdang/Desktop/streaming_batching_end_to_end_pipeline/streaming_batching_end_to_end_pipeline/data/2024/green_tripdata_2024-02.parquet...
✔️ Tải thành công: green_tripdata_2024-02.parquet
Tải về green_tripdata_2024-03.parquet vào /Users/tamdang/Desktop/streaming_batching_end_to_end_pipeline/streaming_batching_end_to_end_pipeline/data/2024/green_tripdata_2024-03.parquet...
✔️ Tải thành công: green_tripdata_2024-03.parquet
Tải về green_tripdata_2024-04.parquet vào /Users/tamdang/Desktop/streaming_batching_end_to_end_pipeline/streaming_batching_end_to_end_pipeline/data/2024/green_tripdata_2024-04.parquet...
✔️ Tải thành công: green_tripdata_2024-04.parquet
Tải về green_tripdata_2024-05.parquet vào /Users/tam